previous work of interest: https://www.frontiersin.org/articles/10.3389/fninf.2018.00002/full Front. Neuroinform., 16 February 2018 | https://doi.org/10.3389/fninf.2018.00002 Extremely Scalable Spiking Neuronal Network Simulation Code: From Laptops to Exascale Computers

![](https://www.frontiersin.org/files/Articles/317068/fninf-12-00002-HTML-r1/image_m/fninf-12-00002-g004.jpg)

# balanced network

Let's start with a rather complete example modified from 

http://neuralensemble.org/docs/PyNN/examples/VAbenchmarks.html


In [1]:
try:
    import pyNN.spiNNaker as sim
    simulator = 'spinnaker'
except ModuleNotFoundError:
    import pyNN.nest as sim
    simulator = 'nest'

CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH


/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/__init__.py:53: UserWarning:Unable to install NEST extensions. Certain models may not be available.



## initialization

In [2]:
# === Define parameters ========================================================

threads  = 1
rngseed  = 98765
parallel_safe = True

r_ei     = 4.0   # number of excitatory cells:number of inhibitory cells
pconn    = 0.01  # connection probability
stim_dur = 50.   # (ms) duration of random stimulation
rate     = 10.  # (Hz) frequency of the random stimulation

dt       = 0.1   # (ms) simulation timestep
delay    = 0.2
# Cell parameters
area     = 20000. # (µm²)
tau_m    = 20.    # (ms)
cm       = 1.     # (µF/cm²)
g_leak   = 5e-5   # (S/cm²)
E_leak   = -60.  # (mV)
v_thresh = -50.   # (mV)
v_reset  = -60.   # (mV)
t_refrac = 5.     # (ms) (clamped at v_reset)
v_mean   = -60.   # (mV) 'mean' membrane potential, for calculating CUBA weights
tau_exc  = 5.     # (ms)
tau_inh  = 10.    # (ms)

# Synapse parameters
Gexc = 4.     # (nS)
Ginh = 51.    # (nS)
Erev_exc = 0.     # (mV)
Erev_inh = -80.   # (mV)

### what is the synaptic delay???

# === Calculate derived parameters =============================================

area  = area*1e-8                     # convert to cm²
cm    = cm*area*1000                  # convert to nF
Rm    = 1e-6/(g_leak*area)            # membrane resistance in MΩ
assert tau_m == cm*Rm                 # just to check
w_exc    = Gexc*1e-3              # We convert conductances to uS
w_inh    = Ginh*1e-3

# === Build the network ========================================================
extra = {'threads' : threads,
         'filename': "va.xml",
         'label': 'VA'}

cell_params = {
    'tau_m'      : tau_m,    'tau_syn_E'  : tau_exc,  'tau_syn_I'  : tau_inh,
    'v_rest'     : E_leak,   'v_reset'    : v_reset,  'v_thresh'   : v_thresh,
    'cm'         : cm,       'tau_refrac' : t_refrac}

cell_params['e_rev_E'] = Erev_exc
cell_params['e_rev_I'] = Erev_inh


## running it...


In [3]:
def run(N_pop=1000,  # number of cells
        simtime=1000, # (ms) simulaton duration
        n_record=40, 
        do_nixio=False, verbose=False):
    
    simtime = dt * int(simtime/dt)
    n_exc = int(round((N_pop*r_ei/(1+r_ei)))) # number of excitatory cells
    n_inh = N_pop - n_exc                     # number of inhibitory cells

    from pyNN.utility import Timer, ProgressBar, init_logging, normalized_filename
    from pyNN.random import NumpyRNG, RandomDistribution
    from neo.io import NixIO

    init_logging(None, debug=False)

    timer = Timer()
    
    node_id = sim.setup(timestep=dt, min_delay=delay, max_delay=10.0, **extra)
    num_process = sim.num_processes()

    timer.start()

    if verbose: print("%s Creating cell populations..." % node_id)

    # create a single population of neurons, and then use population views to define
    # excitatory and inhibitory sub-populations
    celltype = sim.IF_cond_exp
    all_cells = sim.Population(n_exc + n_inh, celltype(**cell_params), label="All Cells")
    exc_cells = all_cells[:n_exc]
    #exc_cells.label = "Excitatory cells"
    inh_cells = all_cells[n_exc:]
    #inh_cells.label = "Inhibitory cells"

    ext_stim = sim.Population(20, sim.SpikeSourcePoisson(rate=rate, duration=stim_dur), 
                              label="expoisson")
    rconn = 0.01
    ext_conn = sim.FixedProbabilityConnector(rconn)
    ext_syn = sim.StaticSynapse(weight=0.1)

    if verbose: print("%s Initialising membrane potential to random values..." % node_id)
    rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
    uniformDistr = RandomDistribution('uniform', low=v_reset, high=v_thresh, rng=rng)
    all_cells.initialize(v=uniformDistr)

    if verbose: 
        print("%s Connecting populations..." % node_id)
        progress_bar = ProgressBar(width=20)
    else:
        progress_bar = None
    connector = sim.FixedProbabilityConnector(pconn, rng=rng, callback=progress_bar)
    exc_syn = sim.StaticSynapse(weight=w_exc, delay=delay)
    inh_syn = sim.StaticSynapse(weight=w_inh, delay=delay)

    connections = {}
    connections['exc'] = sim.Projection(exc_cells, all_cells, connector, exc_syn, receptor_type='excitatory')
    connections['inh'] = sim.Projection(inh_cells, all_cells, connector, inh_syn, receptor_type='inhibitory')
    connections['ext'] = sim.Projection(ext_stim, all_cells, ext_conn, ext_syn, receptor_type='excitatory')

    # === Setup recording ==========================================================
    if verbose: print("%s Setting up recording..." % node_id)
    all_cells.record('spikes')
    exc_cells[[0, 1]].record('v')

    buildCPUTime = timer.diff()*1000

    # === Run simulation ===========================================================
    if verbose: print("%d Running simulation..." % node_id)
    sim.run(simtime)

    E_count = exc_cells.mean_spike_count()
    I_count = inh_cells.mean_spike_count()

    #n_connections = "%d e→e,i  %d i→e,i" % (connections['exc'].size(),
    #                                      connections['inh'].size())
    simCPUTime = timer.diff()*1000
    # === Output results to file ====================================================

    if verbose: print("%d Writing data to file..." % node_id)

    root, basename, extension = "Results", "VAbenchmarks_exc", "h5" if do_nixio else "pkl"
    
    filename = normalized_filename(root, basename, extension, simulator)
    if do_nixio:
        with NixIO(filename) as io:
            exc_cells.write_data(io)
        with NixIO(filename.replace("exc", "inh")) as io:
            inh_cells.write_data(io)
    else:
        exc_cells.write_data(filename)
        inh_cells.write_data(filename.replace("exc", "inh"))
        
    writeCPUTime = timer.diff()*1000

    if verbose: 
        print("\n--- Vogels-Abbott Network Simulation ---")
        print("Nodes                  : %d" % 1)
        print("Number of Neurons      : %d" % N_pop)
        #print("Number of Synapses     : %s" % n_connections)
        print("Excitatory conductance : %g nS" % Gexc)
        print("Inhibitory conductance : %g nS" % Ginh)
        print("Excitatory rate        : %g Hz" % (E_count * 1000.0 / simtime,))
        print("Inhibitory rate        : %g Hz" % (I_count * 1000.0 / simtime,))
        print("Build time             : %g ms" % buildCPUTime)
        print("Simulation time        : %g ms" % simCPUTime)
        print("Writing time           : %g ms" % writeCPUTime)


    # === Finished with simulator ==================================================

    sim.end()
    
    return buildCPUTime, simCPUTime, writeCPUTime

In [4]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,matplotlib,pyNN,neo,nixio,pandas  -r -g -b

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

numpy     : 1.19.2
matplotlib: 3.3.2
pyNN      : 0.9.5
neo       : 0.8.0
nixio     : 1.5.0b6

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-56-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Hostname: inv-ope-de06

Git hash: f34be83854ac8bf458f81ef778714973cb6e6ef3

Git repo: https://github.com/SpikeAI/2020-11_brainhack_Project7

Git branch: main

